In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import numpy
import sympy
from sympy.solvers import solve
from sympy import diff
from time import sleep
from progressbar import progressbar
from math import e as euler

MAX_ITERATION = 1000
MAX_ITERATION_QUASE_NEWTON = 5000
MAX_ITERATIONITERATION_DICOTOMICA = 5000

In [2]:
def function(x, f):
    if f == 1:
        return 1/2*(pow(x[0], 2) + (pow(x[1], 2)/3))
    if f == 2:
        a = pow(euler, x[0]*x[1]*x[2]*x[3]*x[4])
        return a

In [3]:
def grad_function(x, f, c):
    if f == 1:
        return [x[0] + 2*c*(x[0] + x[1] - 1), (x[1]/3) + 2*c*(x[0] + x[1] - 1)]
    if f == 2:
        k = pow(euler, x[0]*x[1]*x[2]*x[3]*x[4])
        j = x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 - 10
        l = x[1]*x[2] - (5*x[3]*x[4])
        
        g = [(x[1]*x[2]*x[3]*x[4]*k) + (4*c*x[0]*j) + (3*c*x[0]**2),
             (x[0]*x[2]*x[3]*x[4]*k) + (4*c*x[1]*j) + (3*c*x[1]**2) + (2 *c*x[2]*l),
             (x[0]*x[1]*x[3]*x[4]*k) + (4*c*x[2]*j)                 + (2 *c*x[1]*l),
             (x[0]*x[1]*x[2]*x[4]*k) + (4*c*x[3]*j)                 - (10*c*x[4]*l),
             (x[0]*x[1]*x[2]*x[3]*k) + (4*c*x[4]*j)                 - (10*c*x[3]*l)]
        return g

In [4]:
#Passo 4
def function_phi(x, f, c):
    if f == 1:
        c_p = c * pow((x[0] + x[1] - 1), 2)
        phi = function(x, f) + c_p
        return phi
    if f == 2:
        c_p = pow(x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 - 10, 2)
        c_p += pow(x[1]*x[2] - 5*x[3]*x[4],2)
        c_p += pow(pow(x[0], 3) + pow(x[1], 3) + 1, 2)
        c_p = c_p * c
        phi = function(x, f) + c_p
        return phi

In [5]:
def busca_dicotomica(a_min, a_max, e, f, x_i, d_i, c):
    for k in range(MAX_ITERATIONITERATION_DICOTOMICA):
        if abs(a_max/a_min) < 1 + e*10 and k > 5:
                break
                
        lambda_k = ((a_min+a_max)/2)-e
        lambda_x = [x_i[j] + d_i[j] * lambda_k for j in range(len(x_i))]
        f_lambda = function_phi(lambda_x, f, c)
        
        mi_k = ((a_min+a_max)/2)+e
        mi_x = [x_i[j] + d_i[j] * mi_k for j in range(len(x_i))]
        f_mi = function_phi(mi_x, f, c)
            
        if f_lambda < f_mi:
            a_max = mi_k
        else:
            a_min = lambda_k
    return (a_min+a_max)/2

In [6]:
def DFP(input_D_i, input_s_i, input_y_i):
    s_i = numpy.array([[i] for i in input_s_i])
    y_i = numpy.array([[i] for i in input_y_i])
    D_i = numpy.array(input_D_i)
    p1 = numpy.true_divide(s_i.dot(s_i.transpose()), s_i.transpose().dot(y_i))
    div_p2 = y_i.transpose().dot(D_i).dot(y_i)
    nom_p2 = D_i.dot(y_i).dot(y_i.transpose()).dot(D_i)
    p2 = numpy.true_divide(nom_p2, div_p2)
    return D_i + p1 - p2

In [7]:
def metodo_quase_newton(x_i, e, f, c):
    #Passo 3
    D_i = numpy.identity(len(x_i))
    for i in range(MAX_ITERATION_QUASE_NEWTON):
        
        #Passo 1
        gradiente = grad_function(x_i, f, c)
        #print(f"gradiente = {gradiente}")
        
        #Passo 4
        d_i = -D_i.dot(gradiente)
        #print(f"d_i = {d_i}")
        
        #Passo 5
        alpha = busca_dicotomica(-1, 1, e/100, f, x_i, d_i, c)
        #print(f"alpha = {alpha}")

        #Passo 6
        antigo_x_i = x_i
        x_i = [x_i[k] + alpha*d_i[k] for k in range(len(x_i))]
        #print(f"i = {i}, x_i = {x_i}, alpha = {alpha}, d_i = {d_i}")
        
        antigo_gradiente = gradiente
        gradiente = grad_function(x_i, f, c)
        #print(f"novo_gradiente = {gradiente}")
        
        #Passo 7
        convergiu = True
        for k in range(len(x_i)):
            if abs(antigo_x_i[k] - x_i[k]) > e:
                convergiu = False
        if convergiu:
            break
            
        #Passo 8
        s_i = [x_i[k] - antigo_x_i[k] for k in range(len(x_i))]
        y_i = [gradiente[k] - antigo_gradiente[k] for k in range(len(gradiente))]
        #print(f"s_i = {s_i}")
        #print(f"y_i = {y_i}")
        
        #Passo 9
        np_s_i = numpy.array(s_i)
        np_y_i = numpy.array(y_i)
        result = numpy.dot(np_s_i.transpose(), np_y_i)
        
        #Passo 9
        if result < 0:
            D_i = numpy.identity(len(x_i))
        
        #Passo 10
        if result >= 0:
            D_i = DFP(D_i, np_s_i, np_y_i)
        
        #print(f"D_i+1 = {D_i}")
        #print(f"result = {result}") 
    #print(x_i)
    return x_i

In [8]:
def metodo_das_penalidades(x_i, e, f, c_0, inc_c_0):
    for i in progressbar(range(MAX_ITERATION)):
        #Passo 5
        antigo_x_i = x_i
        x_i = metodo_quase_newton(x_i, e, f, c_0)
            
        #Passo 6 
        convergiu = True
        for k in range(len(x_i)):
            if abs(antigo_x_i[k] - x_i[k]) > e/100:
                convergiu = False
        if convergiu:
            print(f"Numero de iterações foi o suficiente para convergir")
            break
            
        #Passo 7
        c_0 += inc_c_0
    return x_i

In [9]:
mqn2 = metodo_das_penalidades([-2, 2, 2, -1 , -1], 0.01, 2, 0.015, 0.0001)
print(f"\nFunção 1 - Valor cálculado pelo método das penalidades : {mqn2}")

 42% (421 of 1000) |########             | Elapsed Time: 0:00:13 ETA:   0:00:21C:\Users\chris\AppData\Local\Temp/ipykernel_13736/589574807.py:5: RuntimeWarning: overflow encountered in double_scalars
  a = pow(euler, x[0]*x[1]*x[2]*x[3]*x[4])
100% (1000 of 1000) |####################| Elapsed Time: 0:01:03 Time:  0:01:03



Função 1 - Valor cálculado pelo método das penalidades : [-1.7223631207307528, 1.6022992368407258, 1.968509918324563, -0.7478527661706897, -0.7478527661706917]


In [10]:
x, y = sympy.symbols("x y")
f1 = 1/2*(x*x + ((y*y)/3))

mqn1 = metodo_das_penalidades([0, 0], 0.0001, 1, 100, 100)
sp1 = solve([f1.diff(x), f1.diff(y)], [x, y])
print(f"\nFunção 1 - Valor cálculado pelo método das penalidades : {mqn1}\nValor otimizado : {sp1}")

  2% (24 of 1000) |                      | Elapsed Time: 0:00:00 ETA:   0:00:28

Numero de iterações foi o suficiente para convergir

Função 1 - Valor cálculado pelo método das penalidades : [0.2500971790593921, 0.749854739430764]
Valor otimizado : {x: 0.0, y: 0.0}
